In [8]:
import numpy as np
from copy import deepcopy
import numba as nb
from scipy.stats import norm


In [9]:
# @nb.njit(fastmath=True)
def nb_sum(x):
    n_sum = 0
    for i in range(len(x)):
        n_sum += x[i]
    return n_sum


_ = nb_sum(np.array([2, 2]))


# @nb.njit(parallel=True, fastmath=True)
def get_moments(timesteps, time_means, time_sds, prob_agent_less_player, agent_pdf):
    shape = (time_sds.shape[0], time_means.shape[-1])
    EX_R, EX2_R, EX3_R = np.zeros((shape)), np.zeros((shape)), np.zeros((shape))
    EX_G, EX2_G, EX3_G = np.zeros((shape)), np.zeros((shape)), np.zeros((shape))
    dx = timesteps[1] - timesteps[0]

    for i in nb.prange(len(time_sds)):
        sig_y = time_sds[i]
        xpdf = agent_pdf[i, :]

        for j in range(time_means.shape[-1]):  # Need to loop through every possible decision time mean
            #*Commented out is the old way of doing this bc sc.erfc is recognized by numba, but now I know how to use norm.cdf with numba (which is the same as the error function)
            # xpdf = (1/(sig_x*np.sqrt(2*np.pi)))*np.e**((-0.5)*((timesteps - mu_x)/sig_x)**2) # Pdf of agent, used when getting expected value EX_R, etc.
            # prob_x_less_y = (sc.erfc((mu_x - mu_y[i])/(np.sqrt(2)*np.sqrt(sig_x**2 + sig_y**2))))/2 # Probability of a reaction decision, aka player decides after agent
            # y_integrated = np.empty(len(timesteps),dtype=np.float64)
            # y_inverse_integrated = np.empty(len(timesteps),dtype=np.float64)
            # for k in range(len(timesteps)): # Looping here bc numba_scipy version of sc.erfc can only take float, not an array
            #     t = timesteps[k]
            #     y_integrated[k] = (sc.erfc((t - mu_y[i])/(np.sqrt(2)*sig_y)))/2 # Going from x to infinity is the complementary error function (bc we want all the y's that are greater than x)
            #     y_inverse_integrated[k] = (sc.erfc((mu_y[i] - t)/(np.sqrt(2)*sig_y)))/2 # Swap limits of integration (mu_y[i] - t) now

            mu_y = [j] # Put the timing mean in an easy to use variable,
            prob_x_less_y = prob_agent_less_player[i,j]  # get prob agent is less than player for that specific agent mean (i) and timing mean (j)
            prob_x_greater_y = 1 - prob_x_less_y
            y_integrated = 1 - norm.cdf(timesteps, mu_y, sig_y) # For ALL timesteps, what's the probabilit for every timing mean (from 0 to 2000) that the timing mean is greater than that current timestep
            y_inverse_integrated = 1 - y_integrated

            if prob_x_less_y != 0:
                EX_R[i, j] = nb_sum(timesteps*xpdf*y_integrated)*dx / prob_x_less_y
                EX2_R[i, j] = nb_sum((timesteps - EX_R[i, j]) ** 2*xpdf*y_integrated)*dx / prob_x_less_y  # SECOND CENTRAL MOMENT = VARIANCE
                # EX3_R[i,j] = 0 #np.sum((timesteps-EX_R[i,j])**3*xpdf*y_integrated)*dx/prob_x_less_y # THIRD CENTRAL MOMENT = SKEW
            else:
                EX_R[i, j] = 0
                EX2_R[i, j] = 0  # SECOND CENTRAL MOMENT = VARIANCE
                # EX3_R[i,j] = 0 # THIRD CENTRAL MOMENT = SKEW

            if prob_x_greater_y != 0:
                EX_G[i, j] = nb_sum(timesteps*xpdf*y_inverse_integrated)*dx / prob_x_greater_y
                EX2_G[i, j] = (
                    nb_sum((timesteps - EX_G[i, j]) ** 2*xpdf*y_inverse_integrated)*dx / prob_x_greater_y
                )  # SECOND CENTRAL MOMENT = VARIANCE
                # EX3_G[i,j] = 0#np.sum((timesteps-EX_G[i,j])**3*xpdf*y_inverse_integrated)*dx/prob_x_greater_y # THIRD CENTRAL MOMENT = SKEW
            else:
                EX_G[i, j] = 0
                EX2_G[i, j] = 0  # SECOND CENTRAL MOMENT = VARIANCE
                # EX3_G[i,j] = 0 # THIRD CENTRAL MOMENT = SKEW

    return EX_R, EX2_R, EX3_R, EX_G, EX2_G, EX3_G


In [12]:
def _indicator(x,y):
    if x<y:
        return 1
    else:
        return 0

values_agent = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9]])
probs_agent  = np.array([[0.1, 0.2, 0.4, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0]])
mean_agent = np.sum(values_agent*probs_agent)
std_agent = np.sqrt((values_agent-mean_agent)**2 * probs_agent) # Timing std of the fuck it time for the agent

# Fake player distribution
values_player = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9]])
probs_player = np.array([[0.0, 0.0, 0.1, 0.2, 0.4, 0.2, 0.1, 0.0, 0.0]])
mean_player = np.sum(values_player*probs_player) # Timing mean of the fuck it time for the player
std_player = np.sqrt((values_player-mean_player)**2 * probs_player) # Timing std of the fuck it time for the player

prob_agent_less_player = 0
for i in range(len(values_agent)):
    for j in range(len(values_player)):
        if values_agent.flatten()[i]<values_player.flatten()[j]:
            prob_agent_less_player += probs_agent[i]*probs_player[j]

timesteps = np.array((values_player,values_player))
time_means = np.array([[mean_player]])
prob_agent_less_player = np.array([[prob_agent_less_player]])

In [13]:
get_moments(timesteps, time_means, std_player, prob_agent_less_player, probs_agent)

ValueError: setting an array element with a sequence.